In [1]:
import pandas as pd

# carica il file (usa \t se è TSV)
df = pd.read_csv("/Users/enricozhou/Desktop/abc/out/PeerRead/real/o_accepted_t_theorem_referenced/split0/predictions.tsv", sep="\t")

# conta quante righe totali e quante uniche
print("Totale righe:", len(df))
print("Righe uniche:", len(df.drop_duplicates()))

# righe duplicate
dup = df[df.duplicated()]
print("Numero di righe duplicate:", len(dup))

# se vuoi vedere le prime 10 righe duplicate
print(dup.head(10))


Totale righe: 2048
Righe uniche: 2048
Numero di righe duplicate: 0
Empty DataFrame
Columns: [Unnamed: 0, outcome, treatment, in_dev, in_test, in_train, y0, y1, index, g, q0, q1]
Index: []


In [15]:
import tensorflow as tf

tf_record_file = "../dat/PeerRead/proc/arxiv-all.tf_record"

raw_dataset = tf.data.TFRecordDataset(tf_record_file)

for raw_example in raw_dataset.take(20):  # Legge i primi 5 esempi
    example_proto = tf.train.Example()
    example_proto.ParseFromString(raw_example.numpy())
    print(example_proto)
    print("-" * 50)


features {
  feature {
    key: "abstract_contains_deep"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_embedding"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_neural"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_novel"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_outperform"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_state-of-the-art"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "accepted"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "arxiv"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "avg_length_reference_mention_contexts"
    value {
      float_list {
      

In [17]:
import pandas as pd

def inspect_tfrecord(tf_record_file, n=5):
    raw_dataset = tf.data.TFRecordDataset(tf_record_file)
    rows = []
    '''for raw_example in raw_dataset.take(n):
        ex = tf.train.Example()
        ex.ParseFromString(raw_example.numpy())
        f = ex.features.feature
        rows.append({
            "outcome": f["outcome"].int64_list.value[0] if "outcome" in f else None,
            "treatment": f["treatment"].int64_list.value[0] if "treatment" in f else None,
            "in_train": f["in_train"].int64_list.value[0] if "in_train" in f else None,
            "g": f["g"].float_list.value[0] if "g" in f else None,
            "q0": f["q0"].float_list.value[0] if "q0" in f else None,
            "q1": f["q1"].float_list.value[0] if "q1" in f else None,
            "deep": f["abstract_contains_deep"].int64_list.value[0] if "abstract_contains_deep" in f else None,
            "neural": f["abstract_contains_neural"].int64_list.value[0] if "abstract_contains_neural" in f else None,
        })'''
    for raw_example in raw_dataset.take(1):  # prendi un esempio
        ex = tf.train.Example()
        ex.ParseFromString(raw_example.numpy())
        print(list(ex.features.feature.keys()))

    return pd.DataFrame(rows)

df = inspect_tfrecord("../dat/PeerRead/proc/arxiv-all.tf_record", n=20)
print(df)


['avg_sentence_length', 'venue', 'token_mask', 'arxiv', 'abstract_contains_outperform', 'most_recent_reference_year', 'title_contains_neural', 'num_authors', 'num_ref_to_sections', 'avg_length_reference_mention_contexts', 'accepted', 'num_references', 'abstract_contains_embedding', 'num_ref_to_theorems', 'abstract_contains_neural', 'title_length', 'id', 'num_refmentions', 'num_ref_to_figures', 'num_ref_to_tables', 'num_sections', 'title_contains_embedding', 'abstract_contains_novel', 'title_contains_deep', 'many_split', 'contains_appendix', 'abstract_contains_deep', 'abstract_contains_state-of-the-art', 'num_uniq_words', 'token_ids', 'title_contains_gan', 'num_recent_references', 'num_ref_to_equations', 'year']
Empty DataFrame
Columns: []
Index: []


In [10]:
import tensorflow as tf

tf_record_file = "../dat/PeerRead/proc/peerread_bow_meta.tfrecord"

'''raw_dataset = tf.data.TFRecordDataset(tf_record_file)

for raw_example in raw_dataset.take(5):  # Legge i primi 5 esempi
    example_proto = tf.train.Example()
    example_proto.ParseFromString(raw_example.numpy())
    print(example_proto)'''

treatment_vals = []

for record in tf.data.TFRecordDataset(tf_record_file):
    example = tf.train.Example()
    example.ParseFromString(record.numpy())
    treatment = example.features.feature["treatment"].int64_list.value[0]
    treatment_vals.append(treatment)

treatment_vals = tf.constant(treatment_vals)
print("Num treatment = 0:", tf.reduce_sum(tf.cast(treatment_vals == 0, tf.int32)).numpy())
print("Num treatment = 1:", tf.reduce_sum(tf.cast(treatment_vals == 1, tf.int32)).numpy())


IndexError: list index out of range

In [28]:
import tensorflow as tf

tf_record_file = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"

treatment_vals = []
outcome_vals = []

for record in tf.data.TFRecordDataset(tf_record_file):
    example = tf.train.Example()
    example.ParseFromString(record.numpy())

    # Controlla se le feature esistono
    if "treatment" in example.features.feature:
        treatment = example.features.feature["treatment"].int64_list.value[0]
        treatment_vals.append(treatment)
    if "outcome" in example.features.feature:
        outcome = example.features.feature["outcome"].int64_list.value[0]
        outcome_vals.append(outcome)

treatment_vals = tf.constant(treatment_vals)
outcome_vals = tf.constant(outcome_vals)

print("Num treatment = 0:", tf.reduce_sum(tf.cast(treatment_vals == 0, tf.int32)).numpy())
print("Num treatment = 1:", tf.reduce_sum(tf.cast(treatment_vals == 1, tf.int32)).numpy())

print("Num outcome = 0:", tf.reduce_sum(tf.cast(outcome_vals == 0, tf.int32)).numpy())
print("Num outcome = 1:", tf.reduce_sum(tf.cast(outcome_vals == 1, tf.int32)).numpy())


Num treatment = 0: 9008
Num treatment = 1: 2770
Num outcome = 0: 5681
Num outcome = 1: 6097


In [31]:
import tensorflow as tf

filename = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"

# Definizione schema base: adattalo se i nomi/feature sono diversi
feature_description = {
    "label": tf.io.FixedLenFeature([], tf.int64),
    "features": tf.io.VarLenFeature(tf.int64),   # bag-of-words (sparse)
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

raw_dataset = tf.data.TFRecordDataset(filename)
parsed_dataset = raw_dataset.map(_parse_function)

# Stampa i primi 5 esempi
for i, record in enumerate(parsed_dataset.take(5)):
    label = record["label"].numpy()
    features = record["features"].values.numpy()
    print(f"\nEsempio {i+1}:")
    print("  label:", label)
    print("  n_features:", len(features))
    print("  primi 10 indici:", features[:10])


2025-09-24 22:39:27.981222: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 22:39:27.981243: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 22:39:27.981253: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 22:39:27.981269: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 22:39:27.981280: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Featu

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: label (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [32]:
import tensorflow as tf

filename = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)



features {
  feature {
    key: "beta1"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "bow"
    value {
      float_list {
        value: 0.0625
        value: 0.041666668
        value: 0.010416667
        value: 0.020833334
        value: 0.03125
        value: 0.041666668
        value: 0.020833334
        value: 0.0
        value: 0.0
        value: 0.020833334
        value: 0.020833334
        value: 0.020833334
        value: 0.020833334
        value: 0.010416667
        value: 0.010416667
        value: 0.020833334
        value: 0.010416667
        value: 0.020833334
        value: 0.0
        value: 0.010416667
        value: 0.010416667
        value: 0.010416667
        value: 0.0
        value: 0.010416667
        value: 0.0
        value: 0.0
        value: 0.010416667
        value: 0.010416667
        value: 0.0
        value: 0.0
        value: 0.010416667
        value: 0.0
        value: 0.010416667
        value: 0.0
     

In [33]:
import tensorflow as tf

filename = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"

feature_description = {
    "bow": tf.io.VarLenFeature(tf.float32),
    "beta1": tf.io.FixedLenFeature([], tf.float32),
    "confounding_level": tf.io.FixedLenFeature([], tf.string),
    "index": tf.io.FixedLenFeature([], tf.int64),
    "outcome": tf.io.FixedLenFeature([], tf.int64),
    "treatment": tf.io.FixedLenFeature([], tf.int64),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

raw_dataset = tf.data.TFRecordDataset(filename)
parsed_dataset = raw_dataset.map(_parse_function)

for rec in parsed_dataset.take(1):
    print("Index:", rec["index"].numpy())
    print("Confounding level:", rec["confounding_level"].numpy().decode())
    print("Outcome:", rec["outcome"].numpy())
    print("Treatment:", rec["treatment"].numpy())
    print("Beta1:", rec["beta1"].numpy())
    bow = rec["bow"].values.numpy()
    print("Bag-of-words size:", len(bow))
    print("First 20 bow values:", bow[:20])


Index: 0
Confounding level: 1.0
Outcome: 1
Treatment: 1
Beta1: 1.0
Bag-of-words size: 5000
First 20 bow values: [0.0625     0.04166667 0.01041667 0.02083333 0.03125    0.04166667
 0.02083333 0.         0.         0.02083333 0.02083333 0.02083333
 0.02083333 0.01041667 0.01041667 0.02083333 0.01041667 0.02083333
 0.         0.01041667]


In [29]:
import tensorflow as tf

# path al tuo file
filename = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"

raw_dataset = tf.data.TFRecordDataset(filename)

# conta numero di esempi
count = sum(1 for _ in raw_dataset)
print("Numero di esempi:", count)



Numero di esempi: 11778


In [30]:
import tensorflow as tf

# path al file TFRecord
filename = "../dat/PeerRead/proc/peerread_beta1_bow.tfrecord"

# leggi TFRecord
raw_dataset = tf.data.TFRecordDataset(filename)

# schema delle feature (adatta i tipi ai tuoi dati)
feature_description = {
    "label": tf.io.FixedLenFeature([], tf.int64),   # es. accettato/rifiutato
    "features": tf.io.VarLenFeature(tf.int64),      # es. bag-of-words
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

parsed_dataset = raw_dataset.map(_parse_function)

# stampa i primi 5 esempi
for i, record in enumerate(parsed_dataset.take(5)):
    print(f"\nEsempio {i+1}:")
    for key, value in record.items():
        print(f"{key}:", value.numpy())


2025-09-24 21:17:28.043838: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 21:17:28.044012: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 21:17:28.044019: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 21:17:28.044049: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: label (data type: int64) is required but could not be found.
2025-09-24 21:17:28.044054: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Featu

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: label (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [19]:
raw_dataset = tf.data.TFRecordDataset(filename)
for i, _ in enumerate(raw_dataset):
    if i % 1000 == 0:
        print("Letti:", i)
print("Totale esempi:", i+1)


Letti: 0
Letti: 1000
Letti: 2000
Letti: 3000
Letti: 4000
Letti: 5000
Letti: 6000
Letti: 7000
Letti: 8000
Letti: 9000
Letti: 10000
Letti: 11000
Totale esempi: 11778


In [1]:
import tensorflow as tf

raw_dataset = tf.data.TFRecordDataset("../dat/PeerRead/proc/peerread_beta1_bow.tfrecord")

feature_description = {
    "bow": tf.io.VarLenFeature(tf.float32),
    "index": tf.io.FixedLenFeature([], tf.int64),
    "treatment": tf.io.FixedLenFeature([], tf.int64),
    "outcome": tf.io.FixedLenFeature([], tf.int64),
    "confounding_level": tf.io.FixedLenFeature([], tf.string),
    "beta1": tf.io.FixedLenFeature([], tf.float32),
}

def _parse(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

for i, raw_record in enumerate(raw_dataset.take(5)):
    example = _parse(raw_record)
    print("treatment:", example["treatment"].numpy(),
          "outcome:", example["outcome"].numpy(),
          "confounding_level:", example["confounding_level"].numpy(),
          "beta1:", example["beta1"].numpy())


treatment: 1 outcome: 1 confounding_level: b'1.0' beta1: 1.0
treatment: 0 outcome: 1 confounding_level: b'1.0' beta1: 1.0
treatment: 1 outcome: 1 confounding_level: b'1.0' beta1: 1.0
treatment: 0 outcome: 1 confounding_level: b'1.0' beta1: 1.0
treatment: 0 outcome: 1 confounding_level: b'1.0' beta1: 1.0


In [2]:
import tensorflow as tf

# path al tuo TFRecord unico
filename = "../dat/PeerRead/proc/peerread_all_betas_bow.tfrecord"

# descrizione delle feature che ti aspetti
feature_description = {
    "bow": tf.io.VarLenFeature(tf.float32),
    "index": tf.io.FixedLenFeature([], tf.int64),
    "treatment": tf.io.FixedLenFeature([], tf.int64),
    "outcome": tf.io.FixedLenFeature([], tf.int64),
    "y0": tf.io.FixedLenFeature([], tf.float32),
    "y1": tf.io.FixedLenFeature([], tf.float32),
    "confounding_level": tf.io.FixedLenFeature([], tf.string),
    "beta1": tf.io.FixedLenFeature([], tf.float32),
}

def _parse(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

raw_dataset = tf.data.TFRecordDataset(filename)
parsed_dataset = raw_dataset.map(_parse)

# stampa i primi 3 esempi
for i, ex in enumerate(parsed_dataset.take(3)):
    print("Index:", ex["index"].numpy())
    print("Treatment:", ex["treatment"].numpy(), "Outcome:", ex["outcome"].numpy())
    print("y0:", ex["y0"].numpy(), "y1:", ex["y1"].numpy())
    print("Beta1:", ex["beta1"].numpy(), "Confounding:", ex["confounding_level"].numpy())
    print("----")


Index: 0
Treatment: 1 Outcome: 1
y0: 0.51499546 y1: 0.5768852
Beta1: 1.0 Confounding: b'1.0'
----
Index: 1
Treatment: 0 Outcome: 0
y0: 0.51499546 y1: 0.5768852
Beta1: 1.0 Confounding: b'1.0'
----
Index: 2
Treatment: 1 Outcome: 1
y0: 0.51499546 y1: 0.5768852
Beta1: 1.0 Confounding: b'1.0'
----


In [5]:
import pandas as pd
df = pd.read_csv("../out/PeerRead/bow/predictions.tsv", sep="\t")
print(df.head())
print(df.describe())

   outcome  treatment             g        q0        q1   y0   y1
0        1          1  9.559988e-01  0.918436  0.917964  0.0  1.0
1        0          0  1.264670e-02  0.409762  0.408691  0.0  0.0
2        0          1  3.732647e-01  0.040838  0.039536  1.0  0.0
3        1          0  3.933113e-02  0.766394  0.763530  1.0  0.0
4        0          0  7.408584e-07  0.556341  0.555574  0.0  1.0
            outcome     treatment             g            q0            q1   
count  11778.000000  11778.000000  1.177800e+04  11778.000000  11778.000000  \
mean       0.560706      0.235184  2.178248e-01      0.477679      0.477396   
std        0.496322      0.424132  3.595191e-01      0.278945      0.278944   
min        0.000000      0.000000  5.886128e-11      0.000534      0.000519   
25%        0.000000      0.000000  1.879631e-04      0.231254      0.231384   
50%        1.000000      0.000000  6.282500e-03      0.463918      0.463472   
75%        1.000000      0.000000  2.636340e-01    

In [25]:
import pandas as pd
df = pd.read_csv("../out/PeerRead/atm/beta1/predictions.tsv", sep="\t")
print(df.head())
print(df.describe())


   outcome  treatment         g        q0        q1
0        0          1  0.990739  0.495854  0.495836
1        0          0  0.001297  0.328655  0.328645
2        1          1  0.930034  0.862990  0.863046
3        1          0  0.048249  0.970155  0.970184
4        0          0  0.001290  0.326824  0.326814
            outcome     treatment             g            q0            q1
count  11778.000000  11778.000000  11778.000000  11778.000000  11778.000000
mean       0.501358      0.235184      0.237025      0.498475      0.498475
std        0.500019      0.424132      0.404926      0.242724      0.242743
min        0.000000      0.000000      0.001290      0.326821      0.326810
25%        0.000000      0.000000      0.001290      0.327061      0.327051
50%        1.000000      0.000000      0.001406      0.356624      0.356621
75%        1.000000      0.000000      0.128494      0.553115      0.553150
max        1.000000      1.000000      0.993593      0.998313      0.998316


In [35]:
import tensorflow as tf
import numpy as np

filename = "../dat/PeerRead/proc/peerread_beta5_bow.tfrecord"

feature_description = {
    "bow": tf.io.VarLenFeature(tf.float32),
    "index": tf.io.FixedLenFeature([], tf.int64),
    "treatment": tf.io.FixedLenFeature([], tf.int64),
    "outcome": tf.io.FixedLenFeature([], tf.int64),
    "confounding_level": tf.io.FixedLenFeature([], tf.string),
    "beta1": tf.io.FixedLenFeature([], tf.float32),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

dataset = tf.data.TFRecordDataset(filename).map(_parse_function)

bow_sums, treatment_vals, outcome_vals = [], [], []

for rec in dataset.take(1000):  # controlla i primi 1000 esempi
    bow = tf.sparse.to_dense(rec["bow"]).numpy()
    bow_sums.append(bow.sum())
    treatment_vals.append(rec["treatment"].numpy())
    outcome_vals.append(rec["outcome"].numpy())

print("📊 Sanity check sui primi 1000 esempi")
print(" - Media somma(bow):", np.mean(bow_sums))
print(" - Min somma(bow):", np.min(bow_sums))
print(" - Max somma(bow):", np.max(bow_sums))
print(" - Distribuzione treatment (0/1):", np.bincount(treatment_vals))
print(" - Distribuzione outcome (0/1):", np.bincount(outcome_vals))


📊 Sanity check sui primi 1000 esempi
 - Media somma(bow): 1.0
 - Min somma(bow): 0.9999998
 - Max somma(bow): 1.0000001
 - Distribuzione treatment (0/1): [619 381]
 - Distribuzione outcome (0/1): [394 606]
